# Recommendations

In this section we'll learn how to make listing recommendations using Neo4j. As usual, let's first import some libraries:

In [34]:
from py2neo import Graph
import pandas as pd

pd.set_option('display.max_colwidth', -1)

In [35]:
graph = Graph("bolt://localhost", auth=("neo4j", "neo"))

Sicne we're going to make listing suggestions so let's find some users who have written the most reviews so that we have some data to work with.

In [6]:
popular_users_query = """
MATCH (u:User)
RETURN u.id AS id, u.name AS user, size((u)-[:WROTE]->()) AS reviews
ORDER BY reviews DESC
LIMIT 10
"""

graph.run(popular_users_query).to_data_frame()

,id,reviews,user
0,15355355,59,Karen
1,141581986,58,Salvador
2,111293458,54,Elizabeth
3,39274139,54,Van
4,3973614,53,Christian
5,86126627,52,James
6,17387960,51,Obawole
7,563572,51,Daniel
8,197711,50,J. B.
9,16609485,45,Cliff


We can probably pick anyone from this list but 'Salvador' happens to have written reviews of places that have also been reviewed by other people so we'll use him for our example.

The following query finds the listings that Salvador has reviewed the most:

In [7]:
user_query = """
MATCH (u:User {id: $userId})-[:WROTE]->(review)-[:REVIEWS]->(listing:Listing)-[:IN_NEIGHBORHOOD]->(nh)
RETURN listing.id, listing.name, listing.propertyType, nh.name, count(*) AS times
ORDER BY times DESC
"""

user_id = "141581986"

graph.run(user_query, {"userId": user_id}).to_data_frame()

,listing.id,listing.name,listing.propertyType,nh.name,times
0,14133414,Space to rest near LaGuardia Airport,House,Jackson Heights,22
1,21134697,"Pilots, FA only, 15 min. walk LGA. 3 locations",House,Jackson Heights,9
2,17665781,Female Pilots & Female FA only 15 min walk to LGA.,House,Jackson Heights,7
3,21248963,Female Pilots & Female FA only 15 min walk to LGA.,House,Jackson Heights,7
4,17754072,Bed in Family Home Near LGA Airport,Townhouse,Jackson Heights,4
5,17222454,Sun Room Family Home LGA Airport NO CLEANING FEE,Townhouse,Jackson Heights,3
6,11912865,CrashPadsUSA for Airline Crew. Nightly HOTBEDS,House,Richmond Hill,2
7,16276632,Cozy Room Family Home LGA Airport NO CLEANING FEE,Townhouse,Jackson Heights,2
8,21461663,"SMALL CLEAN ROOM, Easy to Time Square, Airport",House,Rego Park,1
9,16553353,❤️❤️❤️ COZY Place by the Park for ONE ❤️❤️❤️,House,Middle Village,1


It looks like Salvador stays in places near to La Guardia Airport based on the names of the listings. 

The following query finds users who have reviewed the same places as Salvador:

In [ ]:
similar_users_query = """
MATCH (u:User {id: $userId})-[:WROTE]->()-[:REVIEWS]->(listing:Listing),
      (other)-[:WROTE]->()-[:REVIEWS]->(listing)
WHERE u <> other      
WITH other, count(distinct listing) AS commonListings      
RETURN other.id, other.name, commonListings
ORDER BY commonListings DESC
LIMIT 10
"""

user_id = "141581986"

graph.run(similar_users_query, {"userId": user_id}).to_data_frame()

We'll return to these users in the next section.

## Collaborative Filtering

[Collaborative filtering](https://towardsdatascience.com/various-implementations-of-collaborative-filtering-100385c6dfe0) is based on the assumption that people like things similar to other things they like, and things that are liked by other people with similar taste.

<img src="https://cdn-images-1.medium.com/max/1600/1*6_NlX6CJYhtxzRM-t6ywkQ.png" width="500px" />

We can use a simple variant of this approach to find listings that were reviewed by people who stayed in other places that Salvador reviewed:

In [6]:
collabarative_filtering_query = """
MATCH (u:User {id: $userId})-[:WROTE]->()-[:REVIEWS]->(listing:Listing),
      (other)-[:WROTE]->()-[:REVIEWS]->(listing)
WHERE u <> other      
WITH u, other, count(distinct listing) AS commonListings
ORDER BY commonListings DESC
LIMIT 10
MATCH (other)-[:WROTE]->(review)-[:REVIEWS]->(listing)
WHERE not((u)-[:WROTE]->()-[:REVIEWS]->(listing))
RETURN listing, [user in collect(DISTINCT other) | user.name] AS users
ORDER BY size(users) DESC
LIMIT 10
"""

user_id = "141581986"
graph.run(collabarative_filtering_query, {"userId": user_id}).to_data_frame()

,listing,users
0,"{'bedrooms': 1, 'availability365': 343, 'price': 48.0, 'propertyType': 'Townhouse', 'accommodates': 1, 'name': 'Cute Tiny Room Family Home by LGA NO CLEANING FEE', 'id': '18173787', 'bathrooms': 2}","[Renee, Mary, Hikaru, Donald]"
1,"{'bedrooms': 1, 'availability365': 340, 'price': 50.0, 'propertyType': 'Townhouse', 'accommodates': 2, 'name': 'Comfy Room Family Home LGA Airport NO CLEANING FEE', 'id': '5115372', 'bathrooms': 2}","[Luke, Renee, Mary, Sharon]"
2,"{'bedrooms': 1, 'availability365': 164, 'price': 45.0, 'propertyType': 'House', 'accommodates': 2, 'name': 'Walking distance to LaGuardia pvt room', 'cleaningFee': 10.0, 'id': '11618854', 'bathrooms': 1}","[Ellie, Dawn, Hikaru]"
3,"{'bedrooms': 1, 'availability365': 58, 'price': 40.0, 'weeklyPrice': 250.0, 'propertyType': 'House', 'accommodates': 1, 'name': 'JFK 10 & LGA 15 MINUTES A/C PRIVATE BEDROOM', 'id': '7670562', 'bathrooms': 1}","[Luke, Renee, Hikaru]"
4,"{'bedrooms': 1, 'availability365': 134, 'price': 33.0, 'accommodates': 2, 'propertyType': 'House', 'name': 'Private cozy room near LGA airport', 'cleaningFee': 10.0, 'id': '16324410', 'bathrooms': 1}","[Mary, Dawn]"
5,"{'bedrooms': 1, 'availability365': 168, 'weeklyPrice': 240.0, 'price': 45.0, 'propertyType': 'House', 'accommodates': 3, 'name': 'Only Steps away from LaGuardia arpt', 'cleaningFee': 10.0, 'id': '10186192', 'bathrooms': 1}","[Katie, Ellie]"
6,"{'bedrooms': 1, 'availability365': 132, 'price': 50.0, 'accommodates': 2, 'propertyType': 'House', 'name': 'Close by La Guardia airport', 'cleaningFee': 10.0, 'id': '18855980', 'bathrooms': 1}","[Renee, Ellie]"
7,"{'bedrooms': 1, 'availability365': 125, 'price': 45.0, 'propertyType': 'House', 'accommodates': 2, 'name': 'Private room near LGA Airport with queen bed', 'cleaningFee': 10.0, 'id': '16325899', 'bathrooms': 1}","[Mary, Donald]"
8,"{'bedrooms': 1, 'availability365': 365, 'price': 59.0, 'accommodates': 3, 'propertyType': 'House', 'name': 'PRIVATE BED ROOM 12 MINS FROM JFK', 'cleaningFee': 0.0, 'id': '15328242', 'bathrooms': 1}",[Luke]
9,"{'bedrooms': 1, 'availability365': 145, 'price': 40.0, 'propertyType': 'House', 'accommodates': 3, 'name': 'Spacious private room near LGA airport', 'cleaningFee': 10.0, 'id': '16475570', 'bathrooms': 1}",[Mary]


The previous query considered users to be similar to each other if they've written reviews on the same listings, but we could do something more sophisticated. 

We want to work out similar users for each user using a similarity measure (e.g. Jaccard, Cosine, Pearson). These algorithms have a complexity of O(n^2) so let's check how many users we have before we do this:

In [8]:
user_query = """
MATCH (u:User)
RETURN count(*)
"""

graph.run(user_query).to_data_frame()

,count(*)
0,877779


We have more than 800,000 users which would result in more than 770 billion comparisons to work out similarity scores. 
That will take a long time so we need to find a way to reduce the number of comparisons that we need to do.

One way to do this is by clustering our users using the Label Propagation algorithm.
We can then run a similarity algorithm on these small sets of users.

In [9]:
clustering_query = """
CALL algo.labelPropagation(
  "MATCH (u:User) WITH u SKIP {skip} LIMIT {limit} RETURN id(u) AS id",
  "MATCH (u1:User) WITH u1 SKIP {skip} LIMIT {limit} MATCH (u1:User)-[:WROTE]->()-[:REVIEWS]->()<-[:REVIEWS]-()<-[:WROTE]-(u2)
   return id(u1) AS source, id(u2) AS target, count(*) AS weight", "BOTH",
  {graph: "cypher", batchSize: 100, partitionProperty: "partition"}
)
"""

# graph.run(clustering_query).data()

[{'nodes': 877779,
  'iterations': 1,
  'loadMillis': 405487,
  'computeMillis': 5269,
  'writeMillis': 17622,
  'write': True,
  'didConverge': False,
  'weightProperty': 'weight',
  'partitionProperty': 'partition'}]

If you want to run the query in the previous cell you can uncomment the last line, but it takes about 10 minutes to complete, so rather than run that we can load the results from a pre computed CSV file.

In [23]:
load_pre_computed_csv_query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM $csvFile AS row
MATCH (u:User {id: row.userId})
SET u.partition = toInteger(row.partition)
"""

csv_file = "https://github.com/neo4j-contrib/training/raw/master/ml_ai/partitions.csv"
graph.run(load_pre_computed_csv_query, {"csvFile": csv_file}).summary().counters

{'properties_set': 877779}

Let's see how many users we have in each partition:

In [24]:
cluster_query = """
MATCH (u:User)
WHERE exists(u.partition)
RETURN u.partition AS partition, count(*) AS count
ORDER BY count DESC
LIMIT 10
"""

clusters = graph.run(cluster_query).to_table()
clusters

partition,count
89838,17651
68512,16787
159338,12001
44413,10433
143153,9571
97898,6954
54910,6836
71702,5320
126025,5091
139794,4643


Our next step could be to iterate through each of the partitions and calculate the similarity of users in each partition, but rather than doing that we'll just calculate the similarity of people in the same cluster as Salvador.

The following query finds the cluster containing Salvador:

In [31]:
user_query = """
MATCH (u:User {id: $userId})
WITH u.partition AS partition, id(u) AS userId
MATCH (u:User {partition: partition})
RETURN partition, count(*), userId
"""

result = graph.run(user_query, {"userId": user_id}).to_table()
partition, _, user_node_id = result[0]
result

partition,count(*),userId
329748,2539,1264506


Next we'll calculate the similarity of users in that cluster, and then filter the results to show users that are similar to Salvador:

In [36]:
similarity_query = """
MATCH (u:User {partition: $cluster})
MATCH (u)-[:WROTE]->()-[:REVIEWS]->(l)
WITH {item:id(u), categories: collect(distinct id(l))} as userData
WITH collect(userData) as data
CALL algo.similarity.jaccard.stream(data, {similarityCutoff: 0.0})
YIELD item1, item2, count1, count2, intersection, similarity
WHERE item1 = $userNodeId or item2 = $userNodeId
RETURN item1, item2, count1, count2, intersection, similarity
ORDER BY similarity DESC
LIMIT 10
"""

result = graph.run(similarity_query, {"cluster": partition, "userNodeId": user_node_id}).to_table()
result

item1,item2,count1,count2,intersection,similarity
1264506,1585260,10,4,3,0.2727272727272727
1247875,1264506,5,10,3,0.25
1188415,1264506,6,10,3,0.23076923076923078
1247895,1264506,3,10,2,0.18181818181818182
1247711,1264506,3,10,2,0.18181818181818182
1188391,1264506,4,10,2,0.16666666666666666
1264506,1406555,10,4,2,0.16666666666666666
1188381,1264506,4,10,2,0.16666666666666666
1188184,1264506,5,10,2,0.15384615384615385
1264506,1390769,10,5,2,0.15384615384615385


In [40]:
similarity_query = """
MATCH (u:User {partition: $cluster})
MATCH (u)-[:WROTE]->()-[:REVIEWS]->(l)
WITH {item:id(u), categories: collect(distinct id(l))} as userData
WITH collect(userData) as data
CALL algo.similarity.jaccard(data, {similarityCutoff: 0.0, write: false})
YIELD nodes, min, max, mean, stdDev, p25, p50, p75, p90, p95, p99, p999, p100, similarityPairs 
RETURN nodes, min, max, mean, stdDev, p25, p50, p75, p90, p95, p99, p999, p100, similarityPairs
"""

result = graph.run(similarity_query, {"cluster": partition}).to_table()
result

nodes,min,max,mean,stdDev,p25,p50,p75,p90,p95,p99,p999,p100,similarityPairs
2539,0.052631378173828125,1.0000073909759521,0.7444287794290972,0.3154473155266656,0.5000035762786865,1.0000073909759521,1.0000073909759521,1.0000073909759521,1.0000073909759521,1.0000073909759521,1.0000073909759521,1.0000073909759521,231546


In this example we set `write` to `false`, which means that no relationships are created between similar users. If we do want to create relationships between users we may prefer to pass in the `topK` parameter, which will find the `k` most similar users.

Let's run the Jaccard Similarity algorithm again, but this time with `topK=3`, which will find 3 similar users for each user. We'll also set `similarityCutoff=0.2` to filter out users who may have only one listing in common.

In [41]:
similarity_query = """
MATCH (u:User {partition: $cluster})
MATCH (u)-[:WROTE]->()-[:REVIEWS]->(l)
WITH {item:id(u), categories: collect(distinct id(l))} as userData
WITH collect(userData) as data
CALL algo.similarity.jaccard(data, {similarityCutoff: 0.2, topK:3, write: true})
YIELD nodes, similarityPairs, writeProperty, writeRelationshipType
RETURN nodes, similarityPairs, writeProperty, writeRelationshipType
"""

result = graph.run(similarity_query, {"cluster": partition}).to_table()
result

nodes,similarityPairs,writeProperty,writeRelationshipType
2539,7605,score,SIMILAR


Let's see what Salvador's similarity sub graph looks like:

In [54]:
similar_query = """
MATCH path = (:User {id: $userId})-[:SIMILAR]-(other)
RETURN path
"""

graph.run(similar_query, {"userId": user_id}).to_table()

If we run this in the Neo4j browser, and double click on each similar user, we'd see the following visual representation:

<img align="left" src="images/similarity-salvador-listings.svg" width="500px">

If you want to try that query in the Neo4j Browser you'll need to set the `userId` parameter before you run the query. You can do that by running the following command in the Neo4j Browser:

`
:params userId => "141581986"
`

We can now write a query to find out what listings the people who are similar to Salvador have reviewed:

In [42]:
similar_query = """
MATCH (:User {id: $userId})-[:SIMILAR]-(other),
      (other)-[:WROTE]->()-[:REVIEWS]->(listing)
RETURN DISTINCT other.name, listing.id, listing.name
ORDER BY other.name
"""

graph.run(similar_query, {"userId": user_id}).to_table()

other.name,listing.id,listing.name
Dawn,16324410,Private cozy room near LGA airport
Dawn,11618854,Walking distance to LaGuardia pvt room
Dawn,16276632,Cozy Room Family Home LGA Airport NO CLEANING FEE
Dawn,17754072,Bed in Family Home Near LGA Airport
Dawn,21248963,Female Pilots & Female FA only 15 min walk to LGA.
Ellie,14133414,Space to rest near LaGuardia Airport
Ellie,16276632,Cozy Room Family Home LGA Airport NO CLEANING FEE
Ellie,17754072,Bed in Family Home Near LGA Airport
Ellie,10186192,Only Steps away from LaGuardia arpt
Ellie,18855980,Close by La Guardia airport


Now let's see if any of these listings are ones that Salvador hasn't previously stayed at:

In [52]:
recommendations_query = """
MATCH (u:User {id: $userId})-[:SIMILAR]-(other),
      (other)-[:WROTE]->(review)-[:REVIEWS]->(listing)
WHERE not((u)-[:WROTE]->()-[:REVIEWS]->(listing))
RETURN listing.id, listing.name, listing.propertyType, listing.bedrooms, listing.price, collect(DISTINCT other.name) AS people
ORDER BY size(people) DESC
"""

graph.run(recommendations_query, {"userId": user_id}).to_table()

listing.id,listing.name,listing.propertyType,listing.bedrooms,listing.price,people
11618854,Walking distance to LaGuardia pvt room,House,1,45.0,"['Dawn', 'Ellie']"
10186192,Only Steps away from LaGuardia arpt,House,1,45.0,['Ellie']
18855980,Close by La Guardia airport,House,1,50.0,['Ellie']
16601841,Great for La Guardia airport guests.,House,1,40.0,['Scott']
16324410,Private cozy room near LGA airport,House,1,33.0,['Dawn']


We could now make these suggestions to Salvador next time that he's looking for a place to stay.

## Exercise

* Have a look at Salvador's similarity graph in the Neo4j Browser - do the similar users make sense?
* Can you write a query to make recommendations for somebody else in the same cluster as Salvador?
* Can we use the Cosine Similarity procedure (with arrays of numeric properties e.g. number of bathrooms, bedrooms) to calculate listing similarities?